In [ ]:
!pip install transformers datasets torch scikit-learn

In [ ]:
!pip install --upgrade transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcu

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# --- 1. Carregar e Preparar os Dados ---

df = pd.read_csv('dataset_tratado.csv', sep='\t')
print(f"Dataset carregado com sucesso! {len(df)} linhas.")

Dataset carregado com sucesso! 35529 linhas.


In [ ]:
df

,codigo_barras,nome_usuario,tipo_vinculo_usuario,matricula_ou_siape,registro_sistema,assunto,categoria,tipo_material,titulo_completo
0,2012067870,INGRID JUSTINO GOMES,ALUNO DE GRADUAÇÃO,2.020012e+10,189680,Fitopatologia -Plantas Plantas -Doenças Agricu...,Fitopatologia,Livro,Manual de fitopatologia /
1,2013114558,INGRID JUSTINO GOMES,ALUNO DE GRADUAÇÃO,2.020012e+10,182818,Plantas -Nutrição,Plantas,Livro,Nutrição mineral de plantas :princípios e pers...
2,2012036298,CAMILA ABDON MENEZES,ALUNO DE GRADUAÇÃO,2.020001e+10,126087,Prótese total Implantes dentários,Prótese,Livro,Prótese total :convencional e sobre implantes /
3,2012036297,REBECA SILVA RIBEIRO CONFESSOR,ALUNO DE GRADUAÇÃO,2.018010e+10,126087,Prótese total Implantes dentários,Prótese,Livro,Prótese total :convencional e sobre implantes /
4,2018008454,BEATRIZ CLAUDINO DA SILVA,ALUNO DE GRADUAÇÃO,2.020014e+10,245138,Cirurgia bucal maxilofacial Face -Cirurgia,Cirurgia,Livro,Cirurgia oral maxilofacial contemporânea /
...,...,...,...,...,...,...,...,...,...
35524,2015033090,MARIA IZABEL RIBEIRO CAVALCANTE DE OLIVEIRA,ALUNO DE PÓS-GRADUAÇÃO,2.024101e+10,226893,Aprendizagem baseada em problemas (PBL)Metodol...,NaN,Livro,Aprendizagem baseada em problemas (PBL) :uma e...
35525,2016020182,ALLYSON DARLAN MOREIRA DA SILVA,ALUNO DE PÓS-GRADUAÇÃO,2.018102e+10,76603,Nômades Grupos sociais Minorias Mudança social...,NaN,Livro,Sobre o nomadismo :vagabundagens pós-modernas /
35526,L195936,CRISTIAN VIDEL BEZERRA TEIXEIRA,ALUNO DE GRADUAÇÃO,2.021005e+10,64524,Ciência militar,NaN,Livro,A arte da guerra /
35527,L201674,CRISTIAN VIDEL BEZERRA TEIXEIRA,ALUNO DE GRADUAÇÃO,2.021005e+10,73016,Ciência politica Etica,NaN,Livro,O principe :comentado por Napoleão Bonaparte /


In [ ]:
# Remover linhas com campos vazios
df.dropna(subset=['assunto', 'categoria'], inplace=True)
df = df[df['assunto'].str.strip().str.len() > 0]

In [ ]:
df

,codigo_barras,nome_usuario,tipo_vinculo_usuario,matricula_ou_siape,registro_sistema,assunto,categoria,tipo_material,titulo_completo
0,2012067870,INGRID JUSTINO GOMES,ALUNO DE GRADUAÇÃO,2.020012e+10,189680,Fitopatologia -Plantas Plantas -Doenças Agricu...,Fitopatologia,Livro,Manual de fitopatologia /
1,2013114558,INGRID JUSTINO GOMES,ALUNO DE GRADUAÇÃO,2.020012e+10,182818,Plantas -Nutrição,Plantas,Livro,Nutrição mineral de plantas :princípios e pers...
2,2012036298,CAMILA ABDON MENEZES,ALUNO DE GRADUAÇÃO,2.020001e+10,126087,Prótese total Implantes dentários,Prótese,Livro,Prótese total :convencional e sobre implantes /
3,2012036297,REBECA SILVA RIBEIRO CONFESSOR,ALUNO DE GRADUAÇÃO,2.018010e+10,126087,Prótese total Implantes dentários,Prótese,Livro,Prótese total :convencional e sobre implantes /
4,2018008454,BEATRIZ CLAUDINO DA SILVA,ALUNO DE GRADUAÇÃO,2.020014e+10,245138,Cirurgia bucal maxilofacial Face -Cirurgia,Cirurgia,Livro,Cirurgia oral maxilofacial contemporânea /
...,...,...,...,...,...,...,...,...,...
1494,2007023467,FABIANA DA SILVA FRANCA,ALUNO DE GRADUAÇÃO,2.020003e+10,101001,Epidemiologia Indicadores básicos de saúde,Epidemiologia,Livro,Epidemiologia :teoria e prática /
1495,2014061807,FABIANA DA SILVA FRANCA,ALUNO DE GRADUAÇÃO,2.020003e+10,191239,Epidemiologia Saúde pública,Epidemiologia,Livro,"Epidemiologia e saúde :fundamentos, métodos e ..."
1496,2010042835,RUAMA GIOVANA DE OLIVEIRA,ALUNO DE GRADUAÇÃO,2.021006e+10,37727,Álgebra linear,Matemática,Livro,Álgebra linear /
1497,2015008787,MYLLENA ROSANA DE ARAUJO MEDEIROS,ALUNO DE GRADUAÇÃO,2.019010e+10,201796,Química orgânica,Química,Livro,Química orgânica /


In [ ]:
# --- 1. Importações ---
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np


# Remover linhas com campos vazios
df.dropna(subset=['assunto', 'categoria'], inplace=True)
df = df[df['assunto'].str.strip().str.len() > 0]

# Remover classes raras (com apenas 1 exemplo)
print(f"Tamanho original: {len(df)}")
counts = df['categoria'].value_counts()
df = df[df['categoria'].isin(counts[counts > 1].index)].copy()
print(f"Tamanho após remover classes raras: {len(df)}")

# Renomear colunas para o padrão HuggingFace
df = df.rename(columns={'assunto': 'text', 'categoria': 'label'})

# Mapear rótulos para números
labels = df['label'].unique().tolist()
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
df['label'] = df['label'].map(label2id)

# --- 3. Dividir em Treino e Teste ---
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
raw_datasets = DatasetDict({'train': train_dataset, 'test': test_dataset})

# --- 4. Carregar Tokenizer e Modelo ---
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# --- 5. Tokenização ---
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# --- 6. Métricas ---
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f1 = f1_score(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "f1": f1}

# --- 7. Definir Argumentos de Treinamento (versão compatível) ---

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,     # frequência do logging
    save_steps=50,        # salva modelo a cada 50 steps
    save_total_limit=2,
    report_to=[],
)


# --- 8. Criar Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# --- 9. Treinar ---
print("Iniciando o treinamento...")
trainer.train()
print("Treinamento concluído.")

# --- 10. Avaliar ---
print("Resultados no conjunto de teste:")
eval_results = trainer.evaluate()
print(eval_results)

# --- 11. Salvar o Modelo ---
trainer.save_model("meu_modelo_assuntos_bert")
print("Modelo salvo em 'meu_modelo_assuntos_bert'")

# --- 12. Teste Prático ---
from transformers import pipeline

pipe = pipeline("text-classification", model="meu_modelo_assuntos_bert", tokenizer=tokenizer)
novos_assuntos = [
    "Direito Constitucional e administrativo",
    "Anatomia da face para cirurgiões",
    "Introdução à Álgebra Linear e Geometria Analítica"
]

print("\nTestando novos exemplos:")
resultados = pipe(novos_assuntos)
for assunto, resultado in zip(novos_assuntos, resultados):
    print(f"Assunto: '{assunto}' -> Categoria Prevista: '{resultado['label']}' (Score: {resultado['score']:.2f})")


Tamanho original: 1498
Tamanho após remover classes raras: 1472


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Step,Training Loss
10,4.302500
20,4.198600
30,4.026700
40,3.765500
50,3.810300
60,3.559600
70,3.418100
80,3.159900
90,3.015300
100,3.112000


Treinamento concluído.
Resultados no conjunto de teste:


{'eval_loss': 1.6668610572814941, 'eval_accuracy': 0.7186440677966102, 'eval_f1': 0.6432973704879037, 'eval_runtime': 2.1605, 'eval_samples_per_second': 136.544, 'eval_steps_per_second': 17.126, 'epoch': 3.0}
Modelo salvo em 'meu_modelo_assuntos_bert'


Device set to use cuda:0



Testando novos exemplos:
Assunto: 'Direito Constitucional e administrativo' -> Categoria Prevista: 'Direito' (Score: 0.59)
Assunto: 'Anatomia da face para cirurgiões' -> Categoria Prevista: 'Anatomia' (Score: 0.41)
Assunto: 'Introdução à Álgebra Linear e Geometria Analítica' -> Categoria Prevista: 'Matemática' (Score: 0.84)


In [ ]:

from transformers import pipeline

pipe = pipeline("text-classification", model="meu_modelo_assuntos_bert", tokenizer=tokenizer)
novos_assuntos = [
    "Direito Constitucional e administrativo",
    "Anatomia da face para cirurgiões",
    "Introdução à Álgebra Linear e Geometria Analítica",
    "Cálculo",
    "Circuitos eletrônicos -Projetos Projeto de sistemas lógico VHDL (Linguagem descritiva hardware)",
    "VertebradosZoologia"
]

print("\nTestando novos exemplos:")
resultados = pipe(novos_assuntos)
for assunto, resultado in zip(novos_assuntos, resultados):
    print(f"Assunto: '{assunto}' -> Categoria Prevista: '{resultado['label']}' (Score: {resultado['score']:.2f})")


Device set to use cuda:0



Testando novos exemplos:
Assunto: 'Direito Constitucional e administrativo' -> Categoria Prevista: 'Direito' (Score: 0.61)
Assunto: 'Anatomia da face para cirurgiões' -> Categoria Prevista: 'Medicina' (Score: 0.21)
Assunto: 'Introdução à Álgebra Linear e Geometria Analítica' -> Categoria Prevista: 'Matemática' (Score: 0.83)
Assunto: 'Cálculo' -> Categoria Prevista: 'Matemática' (Score: 0.84)
Assunto: 'Circuitos eletrônicos -Projetos Projeto de sistemas lógico VHDL (Linguagem descritiva hardware)' -> Categoria Prevista: 'Engenharia' (Score: 0.44)
Assunto: 'VertebradosZoologia' -> Categoria Prevista: 'Biologia' (Score: 0.06)


In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# --- 1. Carregar o seu Modelo Especialista Treinado ---
# Certifique-se de que o nome da pasta 'meu_modelo_assuntos_bert' está correto.
# device=0 usa a GPU do Colab para acelerar drasticamente a previsão.

print("Carregando seu modelo especialista treinado...")
pipe = pipeline("text-classification", model="meu_modelo_assuntos_bert", device=0)
print("Modelo carregado com sucesso!")


# --- 2. Carregar o seu Dataset Completo ---
# Use o nome do arquivo original que contém tanto os dados rotulados quanto os não rotulados.

df_completo = pd.read_csv('dataset_tratado.csv', sep='\t')
print(f"Dataset completo carregado com {len(df_completo)} linhas.")


# --- 3. Separar os Dados Já Rotulados dos Não Rotulados ---
# Mantém uma cópia dos dados que você rotulou manualmente.
df_rotulado = df_completo[df_completo['categoria'].notna()].copy()

# Seleciona apenas as linhas onde a coluna 'categoria' está vazia (nula).
df_para_prever = df_completo[df_completo['categoria'].isna()].copy()


if not df_para_prever.empty:
    print(f"\nEncontradas {len(df_rotulado)} linhas já rotuladas.")
    print(f"Encontradas {len(df_para_prever)} novas linhas para classificar. Iniciando o processo...")

    # --- 4. Fazer as Previsões ---
    # Pega a lista de 'assunto' das linhas não rotuladas para passar ao modelo.
    lista_assuntos = df_para_prever['assunto'].astype(str).tolist()

    # Usamos um 'batch_size' para processar em lotes, o que é muito mais rápido na GPU.
    # A biblioteca tqdm adiciona uma barra de progresso.
    resultados = []
    for out in tqdm(pipe(lista_assuntos, batch_size=16, truncation=True), total=len(lista_assuntos), desc="Classificando"):
        resultados.append(out)

    # --- 5. Adicionar as Previsões ao DataFrame ---
    # Adiciona os resultados como novas colunas para fácil verificação
    df_para_prever['categoria_prevista'] = [res['label'] for res in resultados]
    df_para_prever['score_confianca'] = [res['score'] for res in resultados]

    # Preenche a coluna 'categoria' original com a previsão do modelo
    df_para_prever['categoria'] = df_para_prever['categoria_prevista']

    # --- 6. Unir Tudo e Salvar o Arquivo Final ---
    # Junta os dados que você rotulou com os dados que o modelo previu
    df_final = pd.concat([df_rotulado, df_para_prever])

    output_filename = 'dataset_FINAL_COMPLETO.csv'
    df_final.to_csv(output_filename, sep=';', index=False, encoding='utf-8-sig')

    print(f"\nPROCESSO DE CLASSIFICAÇÃO CONCLUÍDO!")
    print(f"O arquivo final '{output_filename}' foi salvo com {len(df_final)} linhas totalmente classificadas.")

    print("\n--- Amostra das Novas Previsões ---")
    # Mostra as 10 primeiras previsões feitas pelo modelo
    print(df_para_prever[['assunto', 'categoria_prevista', 'score_confianca']].head(10))

else:
    print("Nenhuma linha com categoria nula foi encontrada. O dataset já parece estar completo.")

Carregando seu modelo especialista treinado...


Device set to use cuda:0


Modelo carregado com sucesso!


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset completo carregado com 35529 linhas.

Encontradas 1499 linhas já rotuladas.
Encontradas 34030 novas linhas para classificar. Iniciando o processo...


Classificando: 100%|██████████| 34030/34030 [00:00<00:00, 3432051.68it/s]



PROCESSO DE CLASSIFICAÇÃO CONCLUÍDO!
O arquivo final 'dataset_FINAL_COMPLETO.csv' foi salvo com 35529 linhas totalmente classificadas.

--- Amostra das Novas Previsões ---
                                                assunto categoria_prevista  \
1499                                Geologia estrutural        Geociências   
1500                                Geologia estrutural        Geociências   
1501                                 Cálculo Matemática         Matemática   
1502              Engenharia Mecânica Estática aplicada         Engenharia   
1503  Psicomotricidade Psicologia do desenvolvimento...         Psicologia   
1504                      Neuroanatomia Sistema nervoso           Anatomia   
1505                    Bausch, Pina,Dança-teatro alemã               Arte   
1506                Dança moderna Movimento (Encenação)               Arte   
1507  Scandella, Domenico, Inquisição Cristãos hereg...            Direito   
1508                                   França -